In [ ]:
import os

base_alert_dir = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system"
os.makedirs(f"{base_alert_dir}/outputs", exist_ok=True)

print("✅ Alert system directories ready at:", base_alert_dir)


✅ Alert system directories ready at: /content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system


In [1]:
import os

os.environ["EMAIL_USER"] = "badabagni25r@gmail.com"
os.environ["EMAIL_PASS"] = "yjcm lhxj kfap lsvq"


In [ ]:
import os, json

config = {
    "enabled_channels": {
        "console": True,
        "email": True,  # enable email alerts now if you want
        "gui": False
    },
    "email": {
        "smtp_server": "smtp.gmail.com",
        "smtp_port": 587,
        "sender": os.getenv("EMAIL_USER"),
        "recipients": ["rohi131279@gmail.com"],
        "username": os.getenv("EMAIL_USER"),
        "password": os.getenv("EMAIL_PASS")
    },
    "suppression_window_seconds": 30,
    "outputs_dir": "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/outputs",
    "log_csv": "alert_log.csv",
    "log_text": "alert_summary.txt",
    "alert_timeout_seconds": 2,
    "max_alerts_per_minute": 60
}

with open("/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/alert_config.json", "w") as f:
    json.dump(config, f, indent=4)

print("✅ Secure alert_config.json created!")


✅ Secure alert_config.json created!


In [2]:
import smtplib
from email.mime.text import MIMEText
import os

sender = os.getenv("EMAIL_USER")
password = os.getenv("EMAIL_PASS")
recipient = sender  # send to yourself for testing

msg = MIMEText("✅ Test alert: Your PPE detection system email alerts are working!")
msg["Subject"] = "PPE Detection Alert Test"
msg["From"] = sender
msg["To"] = recipient

try:
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender, password)
        server.send_message(msg)
    print("📧 Test email sent successfully!")
except Exception as e:
    print("❌ Failed to send email:", e)


📧 Test email sent successfully!


In [ ]:
import textwrap

base_path = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system"

# --- console_alert.py ---
console_code = textwrap.dedent("""
import logging
from datetime import datetime

logger = logging.getLogger("alert_system")
logger.setLevel(logging.INFO)
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)
formatter = logging.Formatter("[%(asctime)s] %(levelname)s: %(message)s")
ch.setFormatter(formatter)
if not logger.handlers:
    logger.addHandler(ch)

def send_console_alert(violation_event):
    msg = f"ALERT - {violation_event['video']} Frame:{violation_event['frame']} Violations:{violation_event['violations']}"
    logger.warning(msg)
    return {"status": "sent", "channel": "console", "ts": datetime.utcnow().isoformat()}
""")

# --- email_alert.py ---
email_code = textwrap.dedent("""
import smtplib
from email.message import EmailMessage
from datetime import datetime

def send_email_alert(violation_event, email_cfg):
    msg = EmailMessage()
    subject = f"PPE Violation: {', '.join(violation_event['violations'])} - {violation_event['video']}"
    body = f\"\"\"PPE Violation detected.
Video: {violation_event['video']}
Frame: {violation_event['frame']}
Violations: {', '.join(violation_event['violations'])}
Timestamp (UTC): {violation_event['timestamp']}
-- Alert System\"\"\"
    msg["From"] = email_cfg["sender"]
    msg["To"] = ", ".join(email_cfg["recipients"])
    msg["Subject"] = subject
    msg.set_content(body)

    try:
        with smtplib.SMTP(email_cfg["smtp_server"], email_cfg["smtp_port"], timeout=10) as server:
            server.starttls()
            server.login(email_cfg["username"], email_cfg["password"])
            server.send_message(msg)
        return {"status": "sent", "channel": "email", "ts": datetime.utcnow().isoformat()}
    except Exception as e:
        return {"status": "failed", "channel": "email", "error": str(e), "ts": datetime.utcnow().isoformat()}
""")

# --- alert_manager.py ---
manager_code = textwrap.dedent("""
import os, json, csv, time
from datetime import datetime, timezone
from collections import defaultdict
from console_alert import send_console_alert
from email_alert import send_email_alert

CONFIG_PATH = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system/alert_config.json"

class AlertManager:
    def __init__(self, config_path=CONFIG_PATH):
        self.config_path = config_path
        self._load_config()
        os.makedirs(self.cfg["outputs_dir"], exist_ok=True)
        self.log_csv_path = os.path.join(self.cfg["outputs_dir"], self.cfg["log_csv"])
        self.log_text_path = os.path.join(self.cfg["outputs_dir"], self.cfg["log_text"])
        self._init_csv()
        self.last_alert_time = defaultdict(lambda: 0)

    def _load_config(self):
        with open(self.config_path, "r") as f:
            self.cfg = json.load(f)

    def _init_csv(self):
        if not os.path.exists(self.log_csv_path):
            with open(self.log_csv_path, "w", newline="") as f:
                csv.writer(f).writerow(["utc_timestamp", "video", "frame", "violations", "channels", "status"])

    def _should_alert(self, key):
        now = time.time()
        last = self.last_alert_time.get(key, 0)
        return (now - last) >= self.cfg.get("suppression_window_seconds", 30)

    def _record_alert(self, event, channels, status):
        ts = datetime.utcnow().replace(tzinfo=timezone.utc).isoformat()
        with open(self.log_csv_path, "a", newline="") as f:
            csv.writer(f).writerow([ts, event["video"], event["frame"], ";".join(event["violations"]), ",".join(channels), status])
        with open(self.log_text_path, "a") as f:
            f.write(f"[{ts}] {event['video']} Frame:{event['frame']} Violations:{','.join(event['violations'])} -> {status}\\n")

    def handle_violation(self, event):
        signature = "|".join(sorted(event["violations"]))
        key = f"{event['video']}::{signature}"
        if not self._should_alert(key):
            return {"status": "suppressed"}

        channels, status = [], "sent"

        if self.cfg["enabled_channels"].get("console", False):
            send_console_alert(event)
            channels.append("console")

        if self.cfg["enabled_channels"].get("email", False):
            send_email_alert(event, self.cfg["email"])
            channels.append("email")

        self.last_alert_time[key] = time.time()
        self._record_alert(event, channels, status)
        return {"status": "sent", "channels": channels}
""")

# Write all files
for fname, code in {
    "console_alert.py": console_code,
    "email_alert.py": email_code,
    "alert_manager.py": manager_code,
}.items():
    with open(os.path.join(base_path, fname), "w") as f:
        f.write(code)

print("✅ Alert system scripts created successfully!")


✅ Alert system scripts created successfully!


In [3]:
!ls -l "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system"

total 10
-rw------- 1 root root  670 Nov  3 15:09 alert_config.json
-rw------- 1 root root 2517 Nov  3 15:12 alert_manager.py
-rw------- 1 root root  605 Nov  3 15:12 console_alert.py
-rw------- 1 root root 1113 Nov  3 15:12 email_alert.py
drwx------ 2 root root 4096 Nov  3 14:48 outputs


In [4]:
import sys
sys.path.append("/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/alert_system")

from alert_manager import AlertManager
from datetime import datetime, timezone

# Initialize Alert Manager
am = AlertManager()

# Create a fake violation event
test_event = {
    "video": "test_video.mp4",
    "frame": 123,
    "violations": ["NO-Hardhat", "NO-Mask"],
    "timestamp": datetime.utcnow().replace(tzinfo=timezone.utc).isoformat()
}

# Send test alert
result = am.handle_violation(test_event)
print("📢 Alert Result:", result)


/tmp/ipython-input-2188336223.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().replace(tzinfo=timezone.utc).isoformat()
[2025-11-04 11:38:35,081] WARNING: ALERT - test_video.mp4 Frame:123 Violations:['NO-Hardhat', 'NO-Mask']


📢 Alert Result: {'status': 'sent', 'channels': ['console', 'email']}


In [7]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.0 MB/s eta 0:00:00


In [14]:
import os
from ultralytics import YOLO
import cv2
from datetime import datetime, timezone
from alert_manager import AlertManager

# Initialize model and alert system
model = YOLO("/content/drive/MyDrive/Project/Dataset/css-data/cross_validation/results_fold3/weights/best.pt")
alert_mgr = AlertManager()

# Input and output paths
video_path = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/testVideos/853867-hd_1920_1080_25fps.mp4"
base_name = os.path.basename(video_path)
output_dir = "/content/drive/MyDrive/Project/Dataset/yolov8_ppe_detection/visual_demo/videos"
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, f"alert_{base_name}")

# Read video
cap = cv2.VideoCapture(video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Setup video writer
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (w, h))

frame_id = 0
print(f"Processing video: {base_name}")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_id += 1
    results = model(frame, verbose=False)

    # Annotate frame with detections
    annotated_frame = results[0].plot()

    # collect all detected class names
    names = model.names
    detections = [names[int(cls)] for cls in results[0].boxes.cls]

    # find rule violations (classes with prefix "NO-")
    violations = [v for v in detections if "NO-" in v]

    # ✅ alert system hook
    if violations:
        event = {
            "video": base_name,
            "frame": frame_id,
            "violations": violations,
            "timestamp": datetime.now(timezone.utc).isoformat()  # fixed UTC warning
        }
        alert_mgr.handle_violation(event)

    # Write annotated frame to output video
    out.write(annotated_frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Video processed and saved at: {output_path}")
print("📁 Alert logs stored in:", alert_mgr.cfg['outputs_dir'])


Processing video: 853867-hd_1920_1080_25fps.mp4


[2025-11-04 12:08:04,469] WARNING: ALERT - 853867-hd_1920_1080_25fps.mp4 Frame:16 Violations:['NO-Hardhat']
[2025-11-04 12:08:16,953] WARNING: ALERT - 853867-hd_1920_1080_25fps.mp4 Frame:36 Violations:['NO-Hardhat', 'NO-Safety Vest', 'NO-Safety Vest']
[2025-11-04 12:08:23,423] WARNING: ALERT - 853867-hd_1920_1080_25fps.mp4 Frame:45 Violations:['NO-Hardhat', 'NO-Safety Vest']
[2025-11-04 12:08:37,057] WARNING: ALERT - 853867-hd_1920_1080_25fps.mp4 Frame:67 Violations:['NO-Safety Vest']
[2025-11-04 12:09:08,995] WARNING: ALERT - 853867-hd_1920_1080_25fps.mp4 Frame:121 Violations:['NO-Mask']
[2025-11-04 12:09:30,281] WARNING: ALERT - 853867-hd_1920_1080_25fps.mp4 Frame:156 Violations:['NO-Safety Vest']
[2025-11-04 12:09:34,359] WARNING: ALERT - 853867-hd_1920_1080_25fps.mp4 Frame:160 Violations:['NO-Mask', 'NO-Mask']
[2025-11-04 12:09:36,770] WARNING: ALERT - 853867-hd_1920_1080_25fps.mp4 Frame:161 Violations:['NO-Mask', 'NO-Safety Vest']
[2025-11-04 12:09:40,793] WARNING: ALERT - 853867-

error: OpenCV(4.12.0) /io/opencv/modules/highgui/src/window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'
